In [3]:
import ase, ase.io
import numpy as np
import os
os.chdir('example_data/')

### Read in some data that has atomic forces, virial stresses

In [4]:
atoms = ase.io.read('hcp_Mg_geomopti_randshear_pm_0.01_product_symm_k0p012.extxyz', index=':') ; print(len(atoms))

500


### Set the regularisation inside the Atoms object
Regularisation strength for the forces (virials) is controlled by regularise (regularise_v), and if the modulated regularisation is less than a specified tolerence it is set as the tolerence value (this makes sure reg. is not set to 0, as this may cause instabilities in ML fitting). 

In [5]:
regularise=0.01
regularise_v = 0.01
configwise=False
tol = 1e-3
tol_v=1e-3
for i, a in enumerate(atoms):
    force_arr = a.get_forces()
    force_max = max(np.linalg.norm(force_arr, axis=1))
    if configwise:
        if force_max < tol:
            atoms[i].info['force_sigma'] = tol
        else:
            atoms[i].info['force_sigma'] = regularise*force_max 
    else:
        force_component_sigma = [tol if j < tol else j for j in regularise*abs(force_arr).flatten()]
        force_component_sigma = np.array(force_component_sigma)
        force_component_sigma = np.reshape(force_component_sigma, [len(a), 3])
        atoms[i].arrays["force_component_sigma"] = force_component_sigma

        
for i, a in enumerate(atoms):
    virials = a.info['virial']
    virial_component_sigma = regularise_v*abs(virials)
    x = [tol_v if j < tol_v else j for j in virial_component_sigma.flatten()]
    atoms[i].info['virial_component_sigma'] = np.array(x).flatten()



### How to write to file

In [6]:
Properties='species:S:1:pos:R:3:force:R:3:force_component_sigma:R:3:'
ase.io.extxyz.write_extxyz('now_with_reg.extxyz', atoms, columns=['symbols', 'positions', 'forces', 'force_component_sigma'])


/usr/local/lib/python3.8/dist-packages/ase/io/extxyz.py:997: UserWarning: write_xyz() overwriting array "forces" present in atoms.arrays with stored results from calculator
  warnings.warn('write_xyz() overwriting array "{0}" present '
